# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# get data file
csv_file = "../output_data/weather_data.csv"

# Read in the csv file
file_df = pd.read_csv(csv_file)
file_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ostrovnoy,68.0531,39.5131,-10.33,95,100,6.69,RU,1611639014
1,Mahébourg,-20.4081,57.7000,28.00,69,20,1.54,MU,1611639015
2,Qaanaaq,77.4840,-69.3632,-16.39,65,0,3.56,GL,1611639017
3,Zhuhai,22.2769,113.5678,26.11,73,20,2.57,CN,1611639003
4,Cape Town,-33.9258,18.4232,21.11,88,20,6.17,ZA,1611638909
...,...,...,...,...,...,...,...,...,...
543,Yabrūd,33.9674,36.6574,2.78,80,0,0.28,SY,1611639647
544,Paraíso,9.8383,-83.8656,21.00,73,0,5.14,CR,1611639649
545,Verāval,20.9000,70.3667,24.60,21,0,3.73,IN,1611639350
546,Novyy Urgal,51.0736,132.5642,-15.58,91,100,0.24,RU,1611639651


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# create variables to hold location and heat weight
locations = file_df[["Latitude", "Longitude"]]
weight = file_df["Humidity"]

In [8]:
# create map
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# add heat layer
fig.add_layer(gmaps.heatmap_layer(locations, weights = weight, max_intensity = 100))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# create new dataframe for cities with ideal weather conditions
holiday_df = file_df.loc[(file_df["Wind Speed"] <= 3) & (file_df["Cloudiness"] == 0) & 
                                    (file_df["Max Temp"] >= 25) & (file_df["Max Temp"] <= 30)].dropna()

holiday_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
247,Puerto Escondido,15.8500,-97.0667,25.20,61,0,1.85,MX,1611639301
502,Chauk,20.8833,94.8167,29.01,29,0,2.37,MM,1611639599
507,Graaff-Reinet,-32.2522,24.5308,26.81,42,0,1.73,ZA,1611639605
514,Loikaw,19.6742,97.2094,29.41,22,0,1.62,MM,1611639613


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# create hotel_df
hotel_df = holiday_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# add new hotel column to dataframe
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
247,Puerto Escondido,MX,15.8500,-97.0667,
502,Chauk,MM,20.8833,94.8167,
507,Graaff-Reinet,ZA,-32.2522,24.5308,
514,Loikaw,MM,19.6742,97.2094,


In [11]:
# set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

In [12]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [14]:
# loop through data frame indexes to get closest hotel
for index, row in hotel_df.iterrows():
    
    # get city name, lat, lnt from df
    city_name = row["City"]
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # add keyword to params dic
    params["location"] = f"{lat},{lng}"

    # complete url and make API request
    print(f"Retrieving Results for: {city_name}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    # try save the hotel name (if there is one) to dataframe
    try:
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, let me know
    except (KeyError, IndexError):
        print("Not available... skipping.")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

Retrieving Results for: Puerto Escondido
The closest hotel in Puerto Escondido is Selina Puerto Escondido.
Retrieving Results for: Chauk
The closest hotel in Chauk is Bo Khin Guest House.
Retrieving Results for: Graaff-Reinet
The closest hotel in Graaff-Reinet is Drostdy Hotel.
Retrieving Results for: Loikaw
The closest hotel in Loikaw is Kayah Land Hotel.


In [15]:
# check dataframe is correct
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
247,Puerto Escondido,MX,15.8500,-97.0667,Selina Puerto Escondido
502,Chauk,MM,20.8833,94.8167,Bo Khin Guest House
507,Graaff-Reinet,ZA,-32.2522,24.5308,Drostdy Hotel
514,Loikaw,MM,19.6742,97.2094,Kayah Land Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))